# Question 3

In [7]:
import torch
import torch.nn as nn
import pandas as pd
import os
from sklearn.preprocessing import LabelEncoder
from torch.utils.data import Dataset
import sklearn
from skimage import io,transform
from torchvision import datasets
from torchvision.transforms import ToTensor

%matplotlib inline
import matplotlib.pyplot as plt

In [8]:
df = pd.read_csv("deeplearning/train/labels.csv",header=None)

In [9]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [20]:
def show_face(image, camera_position):
    """Show image with landmarks"""
    plt.title(camera_position)
    plt.imshow(image,cmap="gray")
    plt.pause(0.001)  # pause a bit so that plots are updated


In [11]:
def encode_dataframe(df):
    le = LabelEncoder()
    df[1] = df[1].astype(str) + df[2].astype(str)+ df[3].astype(str)
    df[1]  = le.fit_transform(df[1])
    return df

In [12]:
encode_dataframe(df)

,0,1,2,3
0,0001.jpg,63,0.000000,0.000000
1,0002.jpg,56,0.000000,0.342020
2,0003.jpg,37,0.000000,0.707107
3,0004.jpg,14,0.000000,1.000000
4,0005.jpg,55,0.000000,-0.342020
...,...,...,...,...
1833,1834.jpg,4,-0.719846,0.642788
1834,1835.jpg,2,-0.397131,0.906308
1835,1836.jpg,6,-0.883022,-0.342020
1836,1837.jpg,10,-0.866025,0.000000


In [46]:
class FaceDataset(Dataset):
    """Custom Fashion MNIST dataset."""

    def __init__(self, csv_file, image_dir,transform = None):
        self.labels_df = pd.read_csv(csv_file,header=None)
        self.labels_df = encode_dataframe(self.labels_df)
        self.image_dir = image_dir

    def __len__(self):
        return len(self.labels_df)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        img_name = os.path.join(self.image_dir,
                                self.labels_df.iloc[idx, 0])
        image = io.imread(img_name)

        camera_position = self.labels_df.iloc[idx, 1]
        print(camera_position)
        if self.transform:
            image = self.transform(image)
        data = {'image': image, 'camera_position': camera_position}




        return data

In [47]:
face_dataset = FaceDataset(csv_file='deeplearning/train/labels.csv',
                           image_dir='deeplearning/train')

In [48]:
fig = plt.figure()

for i in range(len(face_dataset)):
    sample = face_dataset[i]

    print(i, sample['image'].shape, sample['camera_position'].shape)

    ax = plt.subplot(1, 2, i+1)
    plt.tight_layout()
    ax.set_title('Sample #{}'.format(i))
    ax.axis('off')
    show_face(**sample)

    if i == 3:
        plt.show()
        break

63


AttributeError: 

<Figure size 432x288 with 0 Axes>